In [1]:
import pytorch_lightning as pl
import os
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader, random_split
import torchvision
from torchvision import transforms, models
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torchvision.datasets import ImageFolder
import torchmetrics
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import Callback
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger

In [2]:
class FlowersClassifierDataModule(pl.LightningDataModule):
    def __init__(self, batch_size: int=16, data_dir: str = r"C:\Users\matth\OneDrive\Desktop\school\archive (1)\flower_data\flower_data"):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size

        # Augmentation policy for training set
    def setup(self, stage):
        transform_train = transforms.Compose([
              transforms.RandomResizedCrop(size=256),
              transforms.RandomRotation(degrees=15),
              transforms.RandomHorizontalFlip(),
              transforms.CenterCrop(size=224),
              transforms.ToTensor(),
              transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
        ])
        # Preprocessing steps applied to validation and test set.
        transform_test = transforms.Compose([
              transforms.Resize(size=256),
              transforms.CenterCrop(size=224),
              transforms.ToTensor(),
              transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
        ])
        
        tr_data = ImageFolder(self.data_dir + '\\train', transform_train)
        self.test_dataset = ImageFolder(self.data_dir + '\\valid', transform_test)
        
  
    # use 20% of training data for validation (I changed it to 50)
        train_set_size = int(len(tr_data) * 0.05)
        valid_set_size = len(tr_data) - train_set_size

    # split the train set into two
        seed = torch.Generator().manual_seed(42)
        self.train_dataset, self.val_dataset = random_split(tr_data, [train_set_size, valid_set_size], generator=seed)

            
    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, num_workers=9)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, num_workers=9)
    
   # this code runs if only test and train are used, when this is done the validation loop is skipped:
    # def train_dataloader(self):
      #  return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=8)

   # def test_dataloader(self):
      #  return DataLoader(self.test_dataset, batch_size=self.batch_size, num_workers=8)
    
    # Error arises upon adding  
    #def val_dataloader(self):
        # return DataLoader(self.val_dataset, batch_size=self.batch_size, num_workers=8)
    

In [3]:
DATA_DIR = r'C:\Users\matth\OneDrive\Desktop\school\archive (1)\flower_data\flower_data'
Batch_size = 5

In [4]:
class FlowersClassifier(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.criterion =  nn.CrossEntropyLoss()
        self.backbone = models.resnet18(pretrained=True)
        self.linear = nn.Linear(1000, 102)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.linear(x)
        return(x)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.criterion(y_hat, y)
        self.log("mytrain_loss", loss, on_epoch=True, prog_bar=True, logger=True)
        return {'loss': loss, 'log': loss}
        
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.criterion(y_hat, y)
        self.log("val_loss", loss, on_epoch=True, prog_bar=True, logger=True)
        return {'val_loss': loss, "pred": y_hat.detach(), "labels": y}

    def configure_optimizers(self):
        optimizer = optim.SGD(model.parameters(), lr=1e-2)
        return optimizer

In [5]:
model = FlowersClassifier()
datamodule = FlowersClassifierDataModule()

C:\Users\matth\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
C:\Users\matth\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
trainer = pl.Trainer(accelerator="gpu", devices=1, max_epochs=2, log_every_n_steps=16, callbacks=[EarlyStopping(monitor="mytrain_acc_step", mode="min")])

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model, datamodule)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | criterion | CrossEntropyLoss | 0     
1 | backbone  | ResNet           | 11.7 M
2 | linear    | Linear           | 102 K 
-----------------------------------------------
11.8 M    Trainable params
0         Non-trainable params
11.8 M    Total params
47.166    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]